In [1]:
import os
import csv
import pandas as pd

from tqdm import tqdm
from collections import Counter

In [2]:
file_list = os.listdir('./Keystrokes/files/')
file_list.remove('readme.txt')
file_list.remove('metadata_participants.txt')
len(file_list)

168593

In [3]:
# for file in file_list:
#     print(file)
#     df = pd.read_csv('./Keystrokes/files/' + file, sep='\t', encoding="unicode_escape", quoting=csv.QUOTE_NONE)

In [5]:
# preproc files
for file in tqdm(file_list):

    f = open('./Keystrokes/files/' + file, 'r', encoding='windows-1252')

    # read file
    content = f.read()
    f.close()

    # replace \n
    content = content.replace('\t\n\t', '\tnewline\t')
    content = content.replace('\t\t\t', '\ttab\t')

    # save file
    f = open('./Keystroke_prc/' + file, 'w', encoding='windows-1252')
    f.write(content)
    f.close()


100%|██████████| 168593/168593 [03:23<00:00, 827.48it/s]


In [6]:
key_counter = Counter()
failed_files = []
failed_due_to_nan = []
for file in tqdm(file_list):
    try: 
        df = pd.read_csv('./Keystroke_prc/' + file , sep='\t', encoding="windows-1252", quoting=csv.QUOTE_NONE)

        if df['KEYCODE'].isnull().values.any():
            failed_due_to_nan.append((file, df['KEYCODE'].isnull().sum(), len(df)))
            continue

        key_counter.update(df['KEYCODE'].tolist())
    except:
        failed_files.append(file)

100%|██████████| 168593/168593 [04:31<00:00, 621.68it/s]


In [7]:
print("Failed due to nan",  len(failed_due_to_nan))
print("Failed due to other", len(failed_files))

Failed due to nan 0
Failed due to other 0


In [11]:
key_counter.most_common()
key_presses_total = sum(key_counter.values())
key_presses_total

123108012

In [ ]:
from key_codes_js import key_codes_js

# key_counter

sum_of_keys = sum(key_counter.values())
sum_of_keys

In [ ]:
df.head()

In [ ]:
key_counter = Counter()


In [ ]:
# Transform to sequence dataset
# Path: 136M_eda.ipynb
grp_df = df.groupby(['PARTICIPANT_ID', 'TEST_SECTION_ID'])

# iterate over each group
for name, seq_group in grp_df:
    print("Participant ID: ", name[0])
    print("Test Section ID: ", name[1])
    # iterate through seq_group
    sequence = []
    press_code = 1
    release_code = 0
    for index, row in seq_group.iterrows():
        press_time = row['PRESS_TIME']
        release_time = row['RELEASE_TIME']
        keycode = row['KEYCODE']
        sequence.append([press_time, press_code, keycode])
        sequence.append([release_time, release_code, keycode])

    # sort sequence by timea
    sequence.sort(key=lambda x: x[0])

    print(sequence)

    # get typing features
    # typing_features = session_to_features(sequence)

    break



#### Ways to improve
Use a different feature selection/feature engineering method.


Current metric treats the pressings as bigrams. Why not tri-grams or n-grams?